<div style="text-align: right">INFO 6105 Data Science Eng Methods and Tools, Lecture 12 Day 2</div>
<div style="text-align: right">Dino Konstantopoulos, 1 April 2020</div>

# Caterpillar brain

<br />
<center>
<img src =ipynb.images/caterpillar.jpg width = 400 />
</center>

So we drew a small brain last time, so now we're going to *design* one! Today is April's fool day, so we're all caterpillars! How many feet do we have? 16!

>Hey, no lauging, *you* try coordinating 16 feet!

Now assume we sense feedback from our 16 feet, and want to decide whether it means we should move *forward* or *backward*. From *experience* we learn that based on the combined input of our feet-sensory neurons, sometimes we should go forward and sometimes backward, and we want to *learn* from that experience because if we don't decide right, we *face-plant*.

So we pop open our caterpillar laptop and fire up Excel, and look at our following spreadsheet:

<br />
<center>
<img src =ipynb.images/excel.png width = 400 />
</center>

It has 17 **features** (columns): The first 16 that correspond to each of our 16 feet, and the last one (the label) says whether, based on the specific input from our feet, we *should* go forward, or *backward*. We wrote our walking history down carefully in our Excel spreadsheet as we are learning how to walk and are still very unsure. But we know how to type 0 and 1 in an Excel spreadheet pretty well! 

Let's assume that we sense only binary input from each one of our feet (i.e. foot touches ground or does not touch ground), and that 1 means go forward and 0 means go backward, and we'd like to *learn* that.

So, how many *independent variables*?

How many *dependent variables*?

How many **observations** (rows)? Well, we don't know that yet!

# Goal

In other words, the goal is to predict one of two states (forward or backward), using a collection of features (feet sensory input) which are all binary. This is called **supervised learning** because we have written down what we *should do* (the label), and we want to train our brain to predict the label based on the 16 sensory features. 

This is also called **discriminative learning**, rather than **generative learning**, because we do not wish to regenerate the initial signal (feet sensory input), but rather we want to predict a 0 (move backwards) or 1 (move forward) in order to avoid a face-plant! So we want to *discriminate* between 0 and 1.

Our graph edges so far have been equal-weighted. But with brains, our edges acquire weights between nodes. It's a bit similar to facebook friends graphs. Our facebook friends are simiarly weighted, but in reality we like some friends a lot more than others!

The prediction model in our caterpillar brain is simple: We assign weights to each feature (foot sensory input). To predict movement forward or backward based on an observation, we check all features that are *active* (foot is on the ground, or = 1) and sum up the weights assigned to these features. If the total is *above* a certain threshold, the result is interpreted as *move forward*, otherwise it’s *move backwards*. 

So our brain is a graph and we initialize edge weights $w_1 = w_2 = \cdots = w_n = 1$.

Then we iterate on each observation of sensory input from all of our feet, consisting of a vector of dimension $n$: $ = [x_1, x_2, \cdots, x_n]$, where $x_i$ corresponds to foot *i* sensory input.

How do we learn? With a **loop**! We loop through our foot-sensory observations (rows) and predict (for each iteration, or **epoch**), using some kind of linear algebra linear map transformation on our neural weights if our brain should output 1 *(go forward)*, or 0 *(go backward)*.

Then, we look up the label corresponding to that foot-sensory observation (we wrote down whether, based on a specific foot-sensory observation for each one of our feet, we should go forward or backward), and we update the weights **only if we make a mistake**:
- **False-positive** error (we predicted 0, go backwards, whereas the label is 1 or go forward): Then for each $x_i == 1$, we set $w_i = 2*w_i$.
- **False-negative** error (we predicted 1, go forward, whereas the label is 0 or go backward): Then for each $x_i == 1$, we set $wi = wi/2$.

Here is the *machine learning* interpretation of the logic above:
If our caterpillar brain predicts walk forward but should predict go backward, it is **over-shooting**, so weights that were used in that prediction (i.e. the weights attached to active features) should be reduced.
Conversely, if the prediction is walk backward but the correct result should be walk forward, the active features are not used enough to reach the threshold, so weights should be bumped up.

Our goal is to minimize the number of face-plants! When we're down to the minimum, we say we have **converged** in our learning, and we can now walk with very few face-plants! We are a *proud* little caterpillar.

<br />
<center>
<img src =ipynb.images/happy-caterpillar.gif width = 400 />
</center>

### Caterpillar algorithm:

In [39]:
import random
from random import randint
import numpy as np

class Caterpillar:
    numInput = 0
    weights = []
    threshold = 0.0
    alpha = 0.0
    
    def __init__(self, numInput, rndSeed):
        self.numInput = numInput
        self.weights = [0] *numInput
        for i in range(len(self.weights)):
            self.weights[i] = numInput / 2.0
        self.threshold = 1.0 * numInput
        self.alpha = 2.0
        random.seed( rndSeed )
        
    # int[] xValues
    def ComputeY(self, xValues):
        sum = 0.0
        for i in range(numInput):
            sum += self.weights[i] * xValues[i]
        if sum > self.threshold:
            return 1
        else:
            return 0
     
    # Fisher-Yates shuffle algorithm int[][] trainData
    def ShuffleObservations(self, trainData):
        for i in range(len(trainData)):
            r = randint(i, len(trainData) - 1)
            tmp = []
            tmp = trainData[r]
            trainData[r] = trainData[i]
            trainData[i] = tmp
         
    # returns double, int[][] trainData
    def Accuracy(self, trainData):
        numCorrect = 0
        numWrong = 0
        xValues = [0] *numInput
        
        for i in range(len(trainData)):
            xValues = np.copy(trainData[i])
            target = trainData[i][numInput] #last value is target
            computed = self.ComputeY(xValues)

            if computed == target:
                numCorrect += 1
            else:
                numWrong += 1
                
        return (numCorrect * 1.0) / (numCorrect + numWrong)
    
    # returns double[], int[][] trainData
    def TrainWeights(self, trainData):
        xValues = [] * numInput
        self.ShuffleObservations(trainData)
        for i in range(len(trainData)):
            #  get the inputs
            xValues = np.copy(trainData[i])
            
            #  last value is target
            target = trainData[i][numInput] 
            
            computed = self.ComputeY(xValues)

            if (computed == 1 and target == 0):
                # need to decrease weight:
                for j in range(numInput):
                    if (xValues[j] == 0): continue
                    self.weights[j] = self.weights[j] / self.alpha #demotion
            elif (computed == 0 and target == 1):
                # need to increase weight:
                for j in range(numInput):
                    if (xValues[j] == 0): continue
                    self.weights[j] = self.weights[j] * self.alpha #promotion

        result = [0.0] *numInput # = number weights
        result = self.weights
        return result

In [40]:
# double[] vector
def ShowVector(vector, decimals, valsPerRow, newLine):
    frmt = '%.' + str(decimals) + 'f'
    for i in range(len(vector)):
        if (i % valsPerRow == 0): print("", end='')
        print(frmt % vector[i] + " ", end='')
    if (newLine): print("")

# int[][] matrix
def ShowMatrix(matrix, decimals, numRows, indices):
    frmt = '%.' + str(decimals) + 'f'
    for i in range(numRows):
        if (indices):
            print("[" + '%02d' % i + "]   ", end='')
        for j in range(len(matrix[i])):
            print(frmt % matrix[i][j] + " ", end='')
        print("")
    lastIndex = len(matrix) - 1
    if (indices):
        print("[" + '%02d' % lastIndex + "]   ", end='')
    for j in range(len(matrix[lastIndex])):
        print(frmt % matrix[lastIndex][j] + " ", end='')
    print("")

In [41]:
import numpy as np

# int[][] data, seed, out int[][] trainData, out int[][] testData
def MakeTrainTest(data, pct, seed):
    totRows = data.shape[0] #compute number of rows in each result
    numTrainRows = int(totRows * pct)
    numTestRows = totRows - numTrainRows
    #trainData = new int[numTrainRows][]
    trainData = np.empty(data.shape)
    #testData = new int[numTestRows][]
    testData = np.empty(data.shape)
    copy = np.empty(data.shape)

    # int[][] copy = new int[data.Length][] #  make a copy of data
    for i in range(copy.shape[0]):
        # by reference to save space
        copy[i] = data[i]
    for i in range(copy.shape[0]):
        # scramble row order of copy
        r = randint(i, copy.shape[0] - 1)
        tmp = copy[r]
        copy[r] = copy[i]
        copy[i] = tmp
    for i in range(numTrainRows):
        # create training
        trainData[i] = copy[i]
    for i in range(numTestRows):
        # create test
        testData[i] = copy[i + numTrainRows]
        
    return trainData, testData

And here is our excel spreadsheeet, already conveniently broken down into a matrix for you!

1st column is sensory input from foot 1,
2nd column is sensory input from foot 2,
3rd column is sensory input from foot 3, 
etc.

All 16 columns are either 1 for foot touches ground or 0 for foot does not touch ground. 

The last column is what our little caterpillar body *should* do: move forward (1), or backward (0). It's the label!

In [42]:
data = np.empty((100,17))

data[0] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ] #  last col is dem = 0, rep = 1
data[1] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[2] =  [ 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0 ]
data[3] =  [ 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0 ]
data[4] =  [ 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0 ]
data[5] =  [ 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0 ]
data[6] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0 ]
data[7] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1 ]
data[8] =  [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[9] =  [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[10] = [ 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1 ]
data[11] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1 ]
data[12] = [ 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0 ]
data[13] = [ 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0 ]
data[14] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1 ]
data[15] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1 ]
data[16] = [ 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0 ]
data[17] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0 ]
data[18] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1 ]
data[19] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[20] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[21] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[22] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[23] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[24] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0 ]
data[25] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0 ]
data[26] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[27] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[28] = [ 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[29] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[30] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[31] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0 ]
data[32] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0 ]
data[33] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[34] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[35] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[36] = [ 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1 ]
data[37] = [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1 ]
data[38] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[39] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0 ]
data[40] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[41] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0 ]
data[42] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0 ]
data[43] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[44] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0 ]
data[45] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0 ]
data[46] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0 ]
data[47] = [ 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[48] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0 ]
data[49] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[50] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[51] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[52] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[53] = [ 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[54] = [ 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0 ]
data[55] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1 ]
data[56] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1 ]
data[57] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ]
data[58] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ]
data[59] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[60] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0 ]
data[61] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[62] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0 ]
data[63] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0 ]
data[64] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0 ]
data[65] = [ 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1 ]
data[66] = [ 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1 ]
data[67] = [ 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[68] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0 ]
data[69] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0 ]
data[70] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0 ]
data[71] = [ 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1 ]
data[72] = [ 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0 ]
data[73] = [ 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1 ]
data[74] = [ 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0 ]
data[75] = [ 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0 ]
data[76] = [ 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0 ]
data[77] = [ 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0 ]
data[78] = [ 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0 ]
data[79] = [ 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[80] = [ 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0 ]
data[81] = [ 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0 ]
data[82] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1 ]
data[83] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[84] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[85] = [ 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0 ]
data[86] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]
data[87] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1 ]
data[88] = [ 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0 ]
data[89] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1 ]
data[90] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0 ]
data[91] = [ 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0 ]
data[92] = [ 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0 ]
data[93] = [ 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0 ]
data[94] = [ 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 ]
data[95] = [ 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0 ]
data[96] = [ 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0 ]
data[97] = [ 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0 ]
data[98] = [ 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0 ]
data[99] = [ 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1 ]

In [43]:
print("First few lines of all data are:")
ShowMatrix(data, 0, 4, True)

First few lines of all data are:
[00]   0 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 
[01]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 
[02]   0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 
[03]   0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 
[99]   0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 


We randomly split an 100-observation excel spreadsheet into an 80-item training set to generate our caterpillar model. A 20-item test set is earmarked to evaluate the accuracy of our model.

In [44]:
print("Splitting data into 80% train and 20% test matrices")
trainData, testData = MakeTrainTest(data, 0.8, 17)

Splitting data into 80% train and 20% test matrices


In [45]:
print("First few rows of training data are:")
ShowMatrix(trainData, 0, 3, True)

First few rows of training data are:
[00]   0 1 0 1 1 1 0 0 0 1 0 1 1 1 0 1 1 
[01]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 
[02]   0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 
[99]   0 0 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 


In [46]:
print("First few rows of testing data are:")
ShowMatrix(testData, 0, 3, True)

First few rows of testing data are:
[00]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 
[01]   0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 0 1 
[02]   1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 
[99]   0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 


In [90]:
trainData, testData = MakeTrainTest(data, 0.8, 17)

In [94]:
%%time

print("Begin training using Winnow algorithm")
numInput = 16
w = Caterpillar(numInput,17) #rndSeed = 0
weights = w.TrainWeights(trainData)
print("Training complete")

Begin training using Winnow algorithm
Training complete
Wall time: 6.98 ms


In [95]:
print("Final model weights are:")
ShowVector(weights, 4, 8, True)

Final model weights are:
0.5000 0.2500 0.1250 8.0000 2.0000 0.1250 2.0000 1.0000 4.0000 2.0000 0.2500 4.0000 2.0000 2.0000 1.0000 0.5000 


In [96]:
trainAcc = w.Accuracy(trainData)
testAcc = w.Accuracy(testData)

print("Prediction accuracy on training data = " + str(trainAcc))
print("Prediction accuracy on test data = " + str(testAcc))

Prediction accuracy on training data = 0.92
Prediction accuracy on test data = 0.81


You should shoot for a training data accuracy above 90%, and a test data accuracy above 80%.

In [50]:
print("Predicting move when all our feet touch ground: ", end='')
yays = [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ]
predicted = w.ComputeY(yays)
if predicted == 0:
    print("move backward!")
else:
    print("forward!")

print("Predicting move when no feet touch ground: ", end='')
nays = [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
predicted2 = w.ComputeY(nays)
if predicted2 == 0:
    print("move backward!")
else:
    print("forward!")


Predicting move when all our feet touch ground: forward!
Predicting move when no feet touch ground: move backawrd!


We assign weights based on each observation of feet sensory input. Some sensory inputs (for some reason) are going to be very discriminative about movement, others much less so. The discriminative ones acquire higher weights. The *frivolous* ones much less.

The algorithm uses only incorrect information. Correct information (correct guess of a label) yields a `nop`. So the algorithm essentially drives the weights of irrelevant predictors to 0, *winnowing them out*. This makes our winnowing classification effective in situations where many of the predictor variables are irrelevant (frivolous). It's a good way to figure out the important (discriminative) factors. 

It is *unclear* how long to train for. We iterate just once through the training data. An alternative would be to 
Iterate multiple times through the training data, stopping after a fixed number of iterations, or when some desired accuracy has been reached.

Success? Believe it or not, **Machine Learning** algorithms are not much different in methodology. Their modeling power is higher due to non-linearities we're going to discuss. But the methodology is very similar:

- Partition data set into 80% observations and 20% test
- Train model with feature observations in order to refine weights (synapse strength)

Once our weights are determined, we can reuse them for any other observation with similar features. Most of the Machine Learning research in the last 20 years was about how to update the weights and what kind of non-linear function to use as a neural transfer function.

# Dataset: Single Proton Emission Computed Tomography data

You know what? I betcha that little caterpillar brain can rival [IBM's Watson](https://www.ibm.com/watson?p1=Search&p4=p50370936620&p5=e&cm_mmc=Search_Google-_-1S_1S-_-WW_NA-_-ibm%20watson_e&cm_mmca7=71700000060771776&cm_mmca8=aud-309367918490:kwd-2846208261&cm_mmca9=Cj0KCQjw1Iv0BRDaARIsAGTWD1uESxxDiNteKuSnd_PrZTJjGvVJKDeqHT5GA5PkomcD6rhlYa8-he0aAjdoEALw_wcB&cm_mmca10=406603549677&cm_mmca11=e&gclid=Cj0KCQjw1Iv0BRDaARIsAGTWD1uESxxDiNteKuSnd_PrZTJjGvVJKDeqHT5GA5PkomcD6rhlYa8-he0aAjdoEALw_wcB&gclsrc=aw.ds) and become a very good doctor!

The Machine learning repository at the University of California, Irvine, has some great data sets. [Here](https://archive.ics.uci.edu/ml/datasets/SPECT+Heart) are data on cardiac Single Proton Emission Computed Tomography (SPECT) images that looks suspiciously similar to our caterpillar data. Each patient is classified into two categories: **normal** and **abnormal**.

- *The dataset describes diagnosing of cardiac Single Proton Emission Computed Tomography (SPECT) images. Each of the patients is classified into two categories: normal and abnormal. The database of 267 SPECT image sets (patients) was processed to extract features that summarize the original SPECT images. As a result, 44 continuous feature pattern was created for each patient. The pattern was further processed to obtain 22 binary feature patterns. The CLIP3 algorithm was used to generate classification rules from these patterns. The CLIP3 algorithm generated rules that were 84.0% accurate (as compared with cardiologists' diagnoses)*. 

SPECT is a good data set for testing ML algorithms; it has 267 instances that are descibed by 23 binary attribute

Our goal is to predict whether a person is categorized normal or abnormal based on 22 binary feature patterns. 

We should strive to ensure that our accuracy on test data is 70% or above (anything approaching 50% is junk: just a guess!).

Let's recall our caterpillar learning algorithm:

Let's import our patient data:

In [97]:
import pandas as pd
data = pd.read_csv("data/SPECT.test")
data.head(10)

,1,1.1,0,0.1,1.2,1.3,0.2,0.3,0.4,1.4,...,0.7,1.6,1.7,1.8,0.8,0.9,1.9,1.10,0.10,0.11
0,1,1,0,0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,1,0,0,1,...,0,1,1,0,0,0,0,0,0,1
2,1,0,1,1,1,0,0,1,0,1,...,1,1,0,1,0,0,0,0,1,0
3,1,0,0,1,0,0,0,0,1,0,...,1,1,0,1,0,0,0,0,0,1
4,1,0,0,1,1,0,1,0,0,1,...,1,0,0,1,0,0,0,0,1,1
5,1,1,0,0,1,0,0,1,1,1,...,1,1,0,1,0,0,0,1,0,1
6,1,1,0,0,1,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
8,1,1,0,0,1,1,1,0,0,1,...,0,1,0,1,1,0,1,0,0,0
9,1,1,0,0,0,1,0,0,0,1,...,1,1,1,0,0,0,1,0,0,0


In [ ]:
print("First few lines of all data are:")
ShowMatrix(data.values, 0, 4, True)

In [ ]:
diagnosis = data['1']
diagnosis.head(10)

Let's drop the label from the dataset:

In [ ]:
data2 = data.drop(labels='1', axis=1)
data2.head(10)

We randomly split an 100-item encoded data set into an 80-item training set to generate the model. A 20-item test set is earmarked to evaluate the accuracy of our model.

In [ ]:
print("Splitting data into 80% train and 20% test matrices")
trainData, testData = MakeTrainTest(data2.values, 0.8, 17)

In [ ]:
print("First few rows of training data are:")
ShowMatrix(trainData, 0, 3, True)

In [ ]:
print("First few rows of testing data are:")
ShowMatrix(testData, 0, 3, True)

Let's *learn* our spectroscopy data:

In [ ]:
%%time

print("Begin training using Caterpillar algorithm")
numInput = 16
w = Caterpillar(numInput, 0) #rndSeed = 0
weights = w.TrainWeights(trainData)
print("Training complete")

In [ ]:
print("Final model weights are:")
ShowVector(weights, 4, 8, True)

In [ ]:
trainAcc = w.Accuracy(trainData)
testAcc = w.Accuracy(testData)

print("Prediction accuracy on training data = " + str(trainAcc))
print("Prediction accuracy on test data = " + str(testAcc))

In [ ]:
print("Predicting diagnosis of patient with all abnormal readings: ", end='')
yays = [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ]
predicted = w.ComputeY(yays)
if predicted == 0:
    print("normal")
else:
    print("abnormal")

print("Predicting diagnosis of patient with all normal readings: ", end='')
nays = [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
predicted2 = w.ComputeY(nays)
if predicted2 == 0:
    print("normal")
else:
    print("abnormal")

# Conclusion

We assign weights based on each observation. Some abnormal features are going to be very discriminative about overll diagnosis, others much less so. The discriminative ones should acquire higher weights. The “*frivolous*” ones much less.

The algorithm uses only incorrect information. Correct information (correct guess of a label) yields a `nop`. So the algorithm essentially drives the weights of irrelevant predictors to 0, *winnowing them out*. This makes Winnow classification effective in situations where many of the predictor variables are irrelevant (frivolous). It's a good way to figure out the important (discriminative) factors. Much like how regression forests told us what doctors should be looking at in correctly diagnosing breast cancer.

See, even a simple caterpillar brain, as shallow as it is (*single layer*!), with a *linear* activation layer can learn a good data model.

Our little Caterpillar model is the *simplest learning graph model* we know, and it has a name: It's called the Winnow.

<br />
<center>
<img src="ipynb.images/winnow.png" width=400 />
</center>

[Winnowing](https://en.wikipedia.org/wiki/Winnowing) means **removing unwanted items**. Its purpose as an algorthm is to train a binary classifier based on binary features, using a *linear* decision boundary.

<br />
<center>
<img src="ipynb.images/winnow-ann.png" width=400 />
    Single-layer activation-less Winnow
</center>

## Idea
Modify the Winnow so that instead of dividing or multiplying by 2, the increase or decrease in the weights is **proportional to the error**. Does this improve accuracy on the same dataset? On any other dataset?